# Staff cleanup #

In [1]:
import pandas as pd

In [2]:
authors = pd.read_csv("authors.csv")
staff = pd.read_csv("staff.csv")

## `authors` ##

In [3]:
del authors["PUBLICATION_ID"]

In [4]:
authors = authors.sort_values("PERSON_ID", axis=0)

In [5]:
auth_pids = authors["PERSON_ID"].values
auth_pids = list(set(auth_pids))

In [6]:
auth_pn = {}
for i in auth_pids:
     auth_pn[i] = list(set(authors[authors["PERSON_ID"]==i]["PUBLISHED_NAME"].values))

In [7]:
auth_pn

{24576: ['SM Shilton', 'Siobhan M Shilton'],
 57346: ['Tim Arnold'],
 24581: ['Becky Conway-Campbell', 'BL Conway-Campbell'],
 49161: ['Lyndsay M-L Coo'],
 24586: ['Joanne Edgar', 'JL Edgar'],
 16395: ['Francesco Mezzadri', 'F Mezzadri'],
 49164: ['C. Howcroft', 'Chris Howcroft'],
 19117: ['KCR Baldock', 'Katherine C R Baldock', 'Katherine C. R. Baldock'],
 16403: ['Emma S. J. Robinson', 'ESJ Robinson', 'Emma S J Robinson'],
 20: ['David Armstrong'],
 24601: ['Pamela M King', 'PM King'],
 16410: ['Siobhan M Mullan',
  'Siobhan Mullan',
  'Siobhan M. Mullan',
  'SM Mullan',
  'S. Mullan'],
 49182: ['Kongtana Trakarnsanga'],
 33: ['SB Barnes', 'Sally B Barnes', 'S B Barnes'],
 8227: ["G.G D'Costa", "Gavin D'Costa", "GG D'Costa", "Gavin G D'Costa"],
 24612: ['JW Harris'],
 24615: ['F Giampapa'],
 32811: ['F Fornetti', 'Francesco Fornetti'],
 44: ['MH Barton'],
 24621: ['TG Harrison', 'Tim G Harrison'],
 1373: ['AG Hoare', 'A G Hoare'],
 24627: ['Stan Zammit', 'Stanley Zammit', 'S Zammit',

## `staff` ##

In [8]:
# Remove TYPE (all are staff)
# Remove START_DATE and END_DATE
del staff["TYPE"]
del staff["START_DATE"]
del staff["END_DATE"]

In [9]:
staff = staff.sort_values("PERSON_ID", axis=0)

In [10]:
staff_pid = list(set(staff["PERSON_ID"].values))

In [11]:
# Check author_ids is a subset of staff_ids
for i in auth_pids:
    if i not in staff_pid:
        print "Person %d not in staff record" % i

In [12]:
new_staff_record = []
for i in staff_pid:
    staff_row = staff[staff["PERSON_ID"]==i]
    j = {}
    j["PERSON_ID"] = i
    j["PUBLISHED_NAME"] =  list(set( list(set(staff_row["PUBLISHED_NAME"].values)) + auth_pn[i]  )) # add published name
    j["FORENAME"] = list(set(staff_row["FORENAME"].values))
    j["SURNAME"] = list(set(staff_row["SURNAME"].values))
    j["ORGANISATION_CODE"] = list(set(staff_row["ORGANISATION_CODE"].values))
    j["JOB_TITLE"] = list(set(staff_row["JOB_TITLE"].values))
    new_staff_record.append(j)

In [13]:
staff_record = pd.DataFrame(new_staff_record)

In [14]:
staff_record.index = staff_record["PERSON_ID"].values
del staff_record["PERSON_ID"]

In [15]:
# reorder columns
staff_record = staff_record[["FORENAME", "SURNAME", "PUBLISHED_NAME", "JOB_TITLE", "ORGANISATION_CODE"]]

In [16]:
# Each person should only have one FORENAME and one SURNAME
for i, row in staff_record.iterrows():
    if len(row["FORENAME"]) != 1:
        print "Two names: %d" % i
    if len(row["SURNAME"]) != 1:
        print "Two surnames: %d" % i
    if len(row["PUBLISHED_NAME"]) != 1:
        print "More than one published names: %d" % i
    if len(row["JOB_TITLE"]) != 1:
        print "More than one job title: %d" % i
    if len(row["ORGANISATION_CODE"]) != 1:
        print "More than one organisation code: %d" % i
    print "----------+++++++++++++--------------"

More than one published names: 24576
More than one job title: 24576
----------+++++++++++++--------------
----------+++++++++++++--------------
More than one published names: 24581
----------+++++++++++++--------------
----------+++++++++++++--------------
More than one published names: 24586
----------+++++++++++++--------------
More than one published names: 16395
----------+++++++++++++--------------
More than one published names: 49164
----------+++++++++++++--------------
----------+++++++++++++--------------
More than one published names: 16403
----------+++++++++++++--------------
----------+++++++++++++--------------
More than one published names: 24601
----------+++++++++++++--------------
More than one published names: 16410
----------+++++++++++++--------------
----------+++++++++++++--------------
----------+++++++++++++--------------
More than one published names: 33
----------+++++++++++++--------------
More than one published names: 8227
----------+++++++++++++----------

In [17]:
# ...cont
for i, row in staff_record.iterrows():
    staff_record.loc[i, "FORENAME"] = row["FORENAME"][0]
    staff_record.loc[i, "SURNAME"] = row["SURNAME"][0]

### Clean job titles ###

In [18]:
## Collect all job titles
job_titles = []
for i in staff_record["JOB_TITLE"].values:
    for j in i:
        if j not in job_titles:
            job_titles.append(j)
print "There are around %d different job titles among 3263 researchers..." % len(job_titles)

There are around 1426 different job titles among 3263 researchers...


In [19]:
# job_titles_stop_words = ["in", "of"]
# job_titles_stems = [
#     "Emeritus Professor",
    
#     "Postdoctoral Senior Research Associate",
#     "Postdoctoral Researcher",
#     "Postdoctoral Fellow",
#     "Postdoctoral Research Associate",
#     "Postdoctoral Training Fellow",    

    
#     "Senior Research Associate",
#     "Senior Research and Teaching Associate",
#     "Senior Lecturer",
#     "Senior Clinical Fellow",
#     "Senior Clinical Lecturer",
#     "Senior Research Fellow",
#     "Senior Teaching Fellow",
#     "Senior Teaching Laboratory Technician",
#     "Senior Histology Laboratory Technician",
#     "Senior Scientific Computer Officer",
#     "Senior Experimental Officer (Research)",
    
#     "Clinical Lecturer",
#     "Clinical Teacher",
#     "Clinical Tutor",
#     "Clinical Teaching Fellow",
    
#     "Visiting Fellow",
        
#     "Teaching Associate",
#     "Teaching Fellow",
#     "Senior Associate Teacher",
#     "Assistant Teacher",
        
#     "Faculty Education Manager",
    
            
#     "Research Fellow",
#     "Research Associate",
#     "Research Collaborator",
#     "Research Manager",
    
    
#     "Technician",
#     "Coordinator",
#     "Co-ordinator",
#     "Director",
#     "Manager",
#     "Project Staff",
#     "Administrator",
#     "Secretary",
    
#     "Chair",
#     "Dean",
#     "Head",
        
#     "Professor",
#     "Reader",
#     "Lecturer",
#     "Tutor"
# ]

# import re
# at = re.compile("^\
# (?:Postdoctoral|Faculty|Visiting)?\s?\
# (?:Emeritus|Senior|Assistant)?\s?\
# (?:Clinical|Centre|Trial)?\s?\
# (?:Research|Teaching|Training|Education|Experimental|Scientific)?\s?\
# (?:Laboratory|Project|Computer|Study)?\s?\
# (?:Associate)?\s?\
# (?:Professor|Reader|Lecturer|Tutor|Dean|Chair|Head|Technician|Coordinator|Co-ordinator|Director|Manager|Staff|Administrator|Secretary|Researcher|Fellow|Teacher|Collaborator|Officer)?\
# \s(?:in|of|-)\s\
# (?:.+)?\
# $",
#                 re.IGNORECASE)

def decompose_title(s, reversed=False):
    titles = [
        ["Postdoctoral","Faculty","Visiting", "Doctoral", "Undergraduate"],
        ["Emeritus","Senior","Assistant"],
        ["Clinical","Centre","Trial", "General", "Nursery","Programme", "Data", "Library", "Dental", "IT", "User", "Team", "Technical", "Systems", "Molecular", "HR", "Building"],
        
        ["Research","Teaching","Training","Education","Experimental","Scientific"],
        ["and Research","and Teaching","and Training","and Education","and Experimental","and Scientific"],
        
        ["Laboratory","Project","Computer","Study", "Switchboard"],
        ["Associate"],
        ["Professor","Reader","Lecturer","Tutor","Dean","Chair","Head","Technician","Coordinator","Co-ordinator","Director","Manager","Staff","Administrator","Secretary","Researcher","Fellow","Teacher","Collaborator","Officer", "Chancelor", "Vice-Chancellor", "Assistant", "Supervisor", "Instructor", "Worker", "Receptionist", "Trainer", "Support", "Bioinformatician", "Administrator", "Leader", "Developer", "Consultant", "Demonstrator", "Librarian", "Accountant", "President", "Student", "Geneticist", "Council", "Adviser", "Surveyor", "Selector", "Operator"],
        ["(Research)"]
    ]
    normalised = []
    
    if reversed:
        titles = titles[::-1]
        s = " ".join(s.split(" ")[::-1])
    for j in titles:
        norm_len = len(normalised)
        for i in j:
            if s.startswith(i):
                s = s.lstrip(i).strip()
                normalised.append(i)
                break
        if norm_len == len(normalised):
            normalised.append("")
    if reversed:
        s = " ".join(s.split(" ")[::-1])
        normalised = normalised[::-1]
    return s.strip(), tuple(normalised)

In [20]:
outs = []
for i in job_titles:
    i = i.replace("/", " ").replace("Lead", "Leader")
    # Forward
    a, an = decompose_title(i)
    # Backward
    if a == i:
        a, an = decompose_title(i, True)
    # Drop words (e.g. XYZ Reader in ABC)
    if a == i:
        changed = False
        a_split = a.split(" ")
        for j in range(1, len(a_split)):
            aa = " ".join(a_split[j:])
            a, an = decompose_title(aa)
            if aa != a:
                changed = True
                break
        if not changed:
            a, an = decompose_title(i)

    outs.append(an)
            
    # Drop title of in at |alt| if extracted something
    #if a == "" or a.startswith("of") or a.startswith("in") or a.startswith("-") or a.startswith(":"):
    if a == i:
        print outs[-1]
print len(outs)

1426


In [21]:
# Set unified job titles
for irow, row in staff_record.iterrows():
    ann = []
    for i in row["JOB_TITLE"]:
        i = i.replace("/", " ").replace("Lead", "Leader")
        # Forward
        a, an = decompose_title(i)
        # Backward
        if a == i:
            a, an = decompose_title(i, True)
        # Drop words (e.g. XYZ Reader in ABC)
        if a == i:
            changed = False
            a_split = a.split(" ")
            for j in range(1, len(a_split)):
                aa = " ".join(a_split[j:])
                a, an = decompose_title(aa)
                if aa != a:
                    changed = True
                    break
            if not changed:
                a, an = decompose_title(i)
        ann.append(an)
    
    staff_record.loc[irow, "JOB_TITLE"] = ann

In [27]:
staff_record.to_pickle("kbs_staff_record.pickle")

In [3]:
staff_record = pd.read_pickle("kbs_staff_record.pickle")

In [4]:
staff_record

,FORENAME,SURNAME,PUBLISHED_NAME,JOB_TITLE,ORGANISATION_CODE
24576,Siobhan,Shilton,"[SM Shilton, Siobhan M Shilton]","[(, , , , , , , Reader, ), (, , , Research, , ...",[FREN]
57346,Tim,Arnold,[Tim Arnold],"[(, , , Research, , , , Collaborator, )]",[CHEM]
24581,Becky,Conway-Campbell,"[Becky Conway-Campbell, BL Conway-Campbell]","[(, , , Research, , , , Fellow, )]",[SOCS]
49161,Lyndsay,Coo,[Lyndsay M-L Coo],"[(, , , , , , , Lecturer, )]",[CLAS]
24586,Joanne,Edgar,"[Joanne Edgar, JL Edgar]","[(, , , Research, , , Associate, , )]",[VESC]
16395,Francesco,Mezzadri,"[F Mezzadri, Francesco Mezzadri]","[(, , , , , , , Professor, )]",[MATH]
49164,Chris,Howcroft,"[C. Howcroft, Chris Howcroft]","[(, , , Research, , , Associate, , )]",[AERO]
10925,Jamie,Jeremy,[JY Jeremy],"[(, Emeritus, , , , , , Professor, )]",[SOCS]
16403,Emma,Robinson,"[Emma S. J. Robinson, Emma S J Robinson, ESJ R...","[(, , , , , , , Reader, )]",[PHPH]
20,David,Armstrong,[David Armstrong],"[(, Emeritus, , , , , , Professor, )]",[PHPH]


In [1]:
staff_record[staff_record["SURNAME"]=="ABC"]

NameError: name 'staff_record' is not defined

In [25]:
staff_record["JOB_TITLE"]

24576    [(, , , , , , , Reader, ), (, , , Research, , ...
57346             [(, , , Research, , , , Collaborator, )]
24581                   [(, , , Research, , , , Fellow, )]
49161                         [(, , , , , , , Lecturer, )]
24586                [(, , , Research, , , Associate, , )]
16395                        [(, , , , , , , Professor, )]
49164                [(, , , Research, , , Associate, , )]
10925                [(, Emeritus, , , , , , Professor, )]
16403                           [(, , , , , , , Reader, )]
20                   [(, Emeritus, , , , , , Professor, )]
24601                   [(, , , Research, , , , Fellow, )]
16410             [(, Senior, , Research, , , , Fellow, )]
29734                   [(Visiting, , , , , , , Fellow, )]
49182                [(, , , Research, , , Associate, , )]
33                           [(, , , , , , , Professor, )]
8227                         [(, , , , , , , Professor, )]
24612                   [(Visiting, , , , , , , Fellow, 